In [2]:
from glob import glob
import numpy as np
import open3d as o3d
import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
id_class = {'building':0, 'vegetation':1, 'ground':2,'undefined':3}
# als_data = glob("Dublin_txt/**.txt")[5:]

In [5]:
als_data = ["Dublin_txt/T_316000_233500_NE_T_316000_233500_SW.txt",
"Dublin_txt/T_316000_233500_NW.txt",
"Dublin_txt/T_316000_233500_SE.txt",
"Dublin_txt/T_316000_234000_NE.txt",
"Dublin_txt/T_316000_234000_NW.txt",
"Dublin_txt/T_316000_234000_SE.txt",
"Dublin_txt/T_316000_234000_SW.txt",
"Dublin_txt/T_316500_234000_SW_T_316500_233500_NW.txt"]


In [7]:
def get_index_global(data,seg_id,index_n_id):
    buff = data[np.where(data[:,6]==seg_id)]
    return buff[np.hstack(tuple(index_n_id[seg_id]["Instance Segmentation"].values())).astype(int)]

In [8]:
def outlier_removal_v1(als_data):
     for idx,_ in tqdm.tqdm(enumerate(als_data)):
          data = np.loadtxt(als_data[idx])
          index_n_id = {}
          for n_id in id_class.values():
               seg_idx = np.where(data[:,6]==n_id)[0]
               dict_ins_id = {}
               for ins_id_unique in np.unique(data[seg_idx][:,7]):
                    # dict_ins_id[ins_id_unique] = np.where(data[seg_idx][:,7]==int(ins_id_unique))[0]
                    buff_ins_id_unique = np.where(data[seg_idx][:,7]==int(ins_id_unique))[0]
                    buff_data_real = data[buff_ins_id_unique]
                    buff_data_real = np.hstack((buff_data_real,buff_ins_id_unique.reshape(len(buff_ins_id_unique),1)))
                    pcd = o3d.geometry.PointCloud()
                    pcd.points =o3d.utility.Vector3dVector(buff_data_real[:,:3])
                    _, ind = pcd.remove_statistical_outlier(nb_neighbors=40,std_ratio=6.0)
                    dict_ins_id[ins_id_unique] = buff_data_real[ind][:,-1]
               index_n_id[n_id] = {"Instance Segmentation":dict_ins_id}

          building = get_index_global(data,0,index_n_id)
          vegetation = get_index_global(data,1,index_n_id)
          ground = get_index_global(data,2,index_n_id)
          undefined = get_index_global(data,3,index_n_id)
          buff_ = np.vstack((building,vegetation,ground,undefined))
          np.savetxt(als_data[idx][:-4]+"_clean.txt",buff_,fmt='%.8f %.8f %.8f %d %d %d %d %d')

In [9]:
outlier_removal_v1(als_data)

8it [1:15:07, 563.38s/it]


In [295]:
# all_count=0
# index_per_class = []
# for i in index_n_id.keys():
#     index_per_class.append(np.hstack(tuple(index_n_id[i]["Instance Segmentation"].values())))
# all_index= np.hstack(tuple(index_per_class))

In [297]:
np.unique(data[all_index.astype(int)][:,6])

array([0.])

In [277]:
# buff_ = data[np.hstack(tuple(index_n_id[0]["Instance Segmentation"].values())).astype(int)]
buff_ = data[all_index.astype(int)]


In [291]:
tuple(index_n_id[1]["Instance Segmentation"].values())

(array([ 867377.,  867378.,  867379., ..., 2144132., 2144133., 2144134.]),
 array([858263., 858264., 858265., ..., 867374., 867375., 867376.]),
 array([789715., 789716., 789717., ..., 858260., 858261., 858262.]),
 array([746025., 746026., 746027., ..., 789712., 789713., 789714.]),
 array([741726., 741727., 741728., ..., 746022., 746023., 746024.]),
 array([702607., 702608., 702609., ..., 741723., 741724., 741725.]),
 array([680292., 680293., 680294., ..., 702604., 702605., 702606.]),
 array([622240., 622241., 622242., ..., 680280., 680281., 680287.]),
 array([433756., 433757., 433758., ..., 622237., 622238., 622239.]),
 array([390358., 390359., 390360., ..., 433753., 433754., 433755.]),
 array([373361., 373362., 373363., ..., 390355., 390356., 390357.]),
 array([346726., 346727., 346728., ..., 373358., 373359., 373360.]),
 array([300527., 300528., 300529., ..., 346723., 346724., 346725.]),
 array([208662., 208663., 208664., ..., 300524., 300525., 300526.]),
 array([ 51120.,  51121.,  5

In [266]:
# np.savetxt(buff_,)
# 
np.savetxt("test.txt",buff_,fmt="%1.9f %1.9f %1.9f %d %d %d %d %d")
# np.savetxt(,,fmt='%.8f %.8f %.8f %d %d %d')

In [146]:
buff_idx = index_n_id[0]["Instance Segmentation"][0] #0[1] = seg id; 0[2] = instance id
buff_data_real = data[buff_idx]
buff_data_real = np.hstack((data_real,buff_idx.reshape(len(buff_idx),1)))

pcd = o3d.geometry.PointCloud()
pcd.points =o3d.utility.Vector3dVector(buff_data_real[:,:3])
_, ind = pcd.remove_statistical_outlier(nb_neighbors=40,std_ratio=6.0)
index_n_id[0]["Instance Segmentation"][0] = buff_data_real[ind][:,-1]

array([6843116., 6843117., 6843118., ..., 7051939., 7051940., 7051941.])

In [270]:
np.unique(buff_[:,6])

array([0.])

In [126]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0, 0, 0])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [127]:
display_inlier_outlier(pcd, ind)

Showing outliers (red) and inliers (gray): 


In [133]:
np.array(pcd.points)[ind] 

array([[3.15971981e+05, 2.34187818e+05, 1.38839998e+01],
       [3.15971980e+05, 2.34187657e+05, 1.54469995e+01],
       [3.15971986e+05, 2.34187660e+05, 1.41669998e+01],
       ...,
       [3.15962717e+05, 2.34181194e+05, 1.69120007e+01],
       [3.15963408e+05, 2.34181097e+05, 1.88750000e+01],
       [3.15964514e+05, 2.34181486e+05, 1.87439995e+01]])